# Data structure and search for potential leakages

Exploratory analysis of the test set compared to the train set, looking at identifying potential leakages that could improve predictions, or simply a specific structure of the test set that we want to reproduce when validating models with train set data.

In [1]:
import pandas as pd
import os

In [2]:
DATA_FOLDER = '../Data/'

sales    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items    = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_cat = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops    = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
test     = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

In [6]:
# transform date column and split day - month - year
sales.loc[:, "date"] = pd.to_datetime(sales["date"], format="%d.%m.%Y")
sales.loc[:, "day"] = sales["date"].dt.day
sales.loc[:, "month"] = sales["date"].dt.month
sales.loc[:, "year"] = sales["date"].dt.year

In [9]:
print("***Train set***")
print("Sales data frame:")
print(sales.shape)
print(sales.nunique())
print("Shops data frame:")
print(shops.shape)
print(shops.nunique())
print("\n*** Test set***")
print(test.shape)
print(test.nunique())

***Train set***
Sales data frame:
(2935849, 9)
date               1034
date_block_num       34
shop_id              60
item_id           21807
item_price        19993
item_cnt_day        198
day                  31
month                12
year                  3
dtype: int64
Shops data frame:
(60, 2)
shop_name    60
shop_id      60
dtype: int64

*** Test set***
(214200, 3)
ID         214200
shop_id        42
item_id      5100
dtype: int64


We observe that the number of rows in the test set corresponds to the product of unique item ID's with unique shop ID's. This hints at how the test set was built: all combinations of items/shops are represented, and we thus expect many zeroes. We'll need to reproduce this structure and augment the train set such that each month has all item/shop combinations that appear in that month.

What are systematic differences between the train and test sets? Are there shops or items that only appear in the test set, for example a shop that opened in November 2015, or a new item launched during that month?

In [3]:
# coverage of train set over test set
def get_coverage(train_set, test_set, key):
    train_unique = train_set[key].unique()
    test_unique  = test_set[key].unique()
    overcoverage = []
    undercoverage = []
    
    for id in train_unique:
        if not id in test_unique:
            overcoverage.append(id)
    
    for id in test_unique:
        if not id in train_unique:
            undercoverage.append(id)
            
    return (overcoverage, undercoverage)
            
overcoverage, undercoverage = get_coverage(sales, test, "shop_id")

print("shop ID's...")
print("...in train set but not in test set:", overcoverage)
print("...in test set but not in train set (argh!):", undercoverage)

overcoverage, undercoverage = get_coverage(sales, test, "item_id")

print("item ID's...")
print("...in train set but not in test set: %s... (n = %s)"%(overcoverage[:10], len(overcoverage)))
print("...in test set but not in train set (argh!): %s... (n = %s)" %(undercoverage[:10], len(undercoverage)))

shop ID's...
...in train set but not in test set: [23, 27, 29, 0, 1, 8, 13, 30, 32, 54, 43, 51, 17, 9, 40, 33, 20, 11]
...in test set but not in train set (argh!): []
item ID's...
...in train set but not in test set: [2552, 2554, 2555, 2564, 2565, 2572, 2573, 2593, 2604, 2609]... (n = 17070)
...in test set but not in train set (argh!): [5320, 5268, 5826, 3538, 3571, 3604, 3407, 3408, 3405, 3984]... (n = 363)


We investigate whether items present in the test set but absent from the train set have enough coverage when looking at categories, so that category information can be leveraged to make predictions for items unknown to the train set.

In [4]:
# category id coverage for missing items in train set
sales_w_category = sales.merge(items, "left", "item_id")
test_w_category = test.merge(items, "left", "item_id")
cat_missing_items = test_w_category[test_w_category["item_id"].isin(undercoverage)].item_category_id.unique()
train_missing = sales_w_category[sales_w_category["item_category_id"].isin(cat_missing_items)]
print(train_missing.item_category_id.value_counts())
print(test_w_category.item_category_id.value_counts())

40    564652
30    351591
55    339585
19    208219
37    192674
23    146789
28    121539
20     79058
63     53845
65     53227
72     47177
75     42603
67     41706
64     37635
70     35484
3      25283
49     23708
31     20649
25     18576
24     14891
58     13702
29     12257
61     12237
56      7931
12      7402
47      5657
15      5360
45      5234
54      5175
7       4459
76      3746
77      3703
42      2835
78      2346
16      2247
44       248
36        19
27         8
0          3
Name: item_category_id, dtype: int64
40    32340
55    28224
37    13902
31    11634
58     9366
      ...  
11       42
79       42
71       42
74       42
0        42
Name: item_category_id, Length: 62, dtype: int64
